In [1]:
from __future__ import print_function, division

from keras.datasets import fashion_mnist,mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 7 * 7, input_dim=self.latent_dim))
        model.add(Reshape((7,7, 512)))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3,strides=1,padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,strides=1, padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, strides=2,kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()
        model.add(Dense(self.img_rows*self.img_cols*self.channels, input_dim=np.prod(self.img_shape)))
        model.add(Reshape((self.img_rows,self.img_cols,self.channels)))

        model.add(Conv2D(64, kernel_size=3, strides=2,padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if  global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_cgan'):
            os.mkdir('images_cgan')
        fig.savefig("images_cgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 128)         0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.159811, acc.: 33.59%] [G loss: 0.593289]
epoch:0 step:2 [D loss: 0.586770, acc.: 59.38%] [G loss: 1.058307]
epoch:0 step:3 [D loss: 0.392995, acc.: 77.34%] [G loss: 1.868736]
epoch:0 step:4 [D loss: 0.269268, acc.: 86.72%] [G loss: 2.396905]
epoch:0 step:5 [D loss: 0.285235, acc.: 87.50%] [G loss: 2.770159]
epoch:0 step:6 [D loss: 0.224948, acc.: 96.09%] [G loss: 2.796331]
epoch:0 step:7 [D loss: 0.209202, acc.: 94.53%] [G loss: 3.706776]
epoch:0 step:8 [D loss: 0.344180, acc.: 85.16%] [G loss: 3.701868]
epoch:0 step:9 [D loss: 0.650198, acc.: 69.53%] [G loss: 3.814477]
epoch:0 step:10 [D loss: 0.801561, acc.: 66.41%] [G loss: 3.672712]
epoch:0 step:11 [D loss: 1.561280, acc.: 39.06%] [G loss: 3.504217]
epoch:0 step:12 [D loss: 1.592679, acc.: 55.47%] [G loss: 4.143368]
epoch:0 step:13 [D loss: 1.051648, acc.: 54.69%] [G loss: 3.840184]
epoch:0 step:14 [D loss: 0.895352, acc.: 66.41%] [G loss: 3.824173]
epoch:0 step:15 [D loss: 0.657380, acc.: 64.06%] [G loss:

epoch:0 step:123 [D loss: 0.093927, acc.: 96.09%] [G loss: 5.000240]
epoch:0 step:124 [D loss: 0.067237, acc.: 97.66%] [G loss: 5.876561]
epoch:0 step:125 [D loss: 0.070229, acc.: 95.31%] [G loss: 7.380005]
epoch:0 step:126 [D loss: 0.110799, acc.: 94.53%] [G loss: 5.615582]
epoch:0 step:127 [D loss: 0.062514, acc.: 98.44%] [G loss: 5.350888]
epoch:0 step:128 [D loss: 0.043432, acc.: 99.22%] [G loss: 4.328786]
epoch:0 step:129 [D loss: 0.252891, acc.: 92.19%] [G loss: 4.514294]
epoch:0 step:130 [D loss: 0.579946, acc.: 82.03%] [G loss: 6.714489]
epoch:0 step:131 [D loss: 0.135614, acc.: 95.31%] [G loss: 6.713301]
epoch:0 step:132 [D loss: 0.126807, acc.: 94.53%] [G loss: 6.226234]
epoch:0 step:133 [D loss: 0.068900, acc.: 97.66%] [G loss: 7.153761]
epoch:0 step:134 [D loss: 0.014945, acc.: 100.00%] [G loss: 7.174509]
epoch:0 step:135 [D loss: 0.113048, acc.: 96.09%] [G loss: 5.933729]
epoch:0 step:136 [D loss: 0.176514, acc.: 89.84%] [G loss: 6.430767]
epoch:0 step:137 [D loss: 0.50681

...

epoch:29 step:27624 [D loss: 0.417432, acc.: 89.06%] [G loss: 1.253079]
epoch:29 step:27625 [D loss: 0.509247, acc.: 82.03%] [G loss: 1.224497]
epoch:29 step:27626 [D loss: 0.381210, acc.: 86.72%] [G loss: 1.247600]
epoch:29 step:27627 [D loss: 0.441881, acc.: 83.59%] [G loss: 1.396441]
epoch:29 step:27628 [D loss: 0.398689, acc.: 84.38%] [G loss: 1.553655]
epoch:29 step:27629 [D loss: 0.397955, acc.: 86.72%] [G loss: 1.988000]
epoch:29 step:27630 [D loss: 0.434083, acc.: 88.28%] [G loss: 1.371949]
epoch:29 step:27631 [D loss: 0.529716, acc.: 68.75%] [G loss: 1.021946]
epoch:29 step:27632 [D loss: 0.664009, acc.: 60.16%] [G loss: 0.942414]
epoch:29 step:27633 [D loss: 0.594985, acc.: 71.09%] [G loss: 2.073890]
epoch:29 step:27634 [D loss: 0.784834, acc.: 46.09%] [G loss: 1.301138]
epoch:29 step:27635 [D loss: 0.904146, acc.: 33.59%] [G loss: 0.910096]
epoch:29 step:27636 [D loss: 0.579394, acc.: 67.97%] [G loss: 0.793295]
epoch:29 step:27637 [D loss: 0.529377, acc.: 75.78%] [G loss: 0.

epoch:29 step:27738 [D loss: 0.585261, acc.: 63.28%] [G loss: 1.668743]
epoch:29 step:27739 [D loss: 0.309632, acc.: 85.16%] [G loss: 1.998192]
epoch:29 step:27740 [D loss: 0.089059, acc.: 97.66%] [G loss: 3.092885]
epoch:29 step:27741 [D loss: 0.457212, acc.: 78.12%] [G loss: 2.304647]
epoch:29 step:27742 [D loss: 0.732156, acc.: 58.59%] [G loss: 2.260274]
epoch:29 step:27743 [D loss: 0.528419, acc.: 75.00%] [G loss: 2.034586]
epoch:29 step:27744 [D loss: 0.566212, acc.: 67.19%] [G loss: 1.979021]
epoch:29 step:27745 [D loss: 0.243014, acc.: 91.41%] [G loss: 1.852776]
epoch:29 step:27746 [D loss: 0.219571, acc.: 94.53%] [G loss: 1.814567]
epoch:29 step:27747 [D loss: 0.463940, acc.: 81.25%] [G loss: 1.916081]
epoch:29 step:27748 [D loss: 0.229707, acc.: 92.19%] [G loss: 2.087037]
epoch:29 step:27749 [D loss: 0.145652, acc.: 98.44%] [G loss: 3.189979]
epoch:29 step:27750 [D loss: 0.163961, acc.: 95.31%] [G loss: 1.738885]
epoch:29 step:27751 [D loss: 0.109130, acc.: 100.00%] [G loss: 2

epoch:29 step:27854 [D loss: 0.475790, acc.: 84.38%] [G loss: 1.130550]
epoch:29 step:27855 [D loss: 0.757329, acc.: 52.34%] [G loss: 0.987153]
epoch:29 step:27856 [D loss: 0.561877, acc.: 69.53%] [G loss: 0.698081]
epoch:29 step:27857 [D loss: 0.620557, acc.: 62.50%] [G loss: 1.041022]
epoch:29 step:27858 [D loss: 0.553499, acc.: 68.75%] [G loss: 1.017331]
epoch:29 step:27859 [D loss: 0.534498, acc.: 77.34%] [G loss: 1.057580]
epoch:29 step:27860 [D loss: 0.513904, acc.: 75.78%] [G loss: 1.335413]
epoch:29 step:27861 [D loss: 0.764967, acc.: 48.44%] [G loss: 1.089604]
epoch:29 step:27862 [D loss: 0.627362, acc.: 68.75%] [G loss: 0.839404]
epoch:29 step:27863 [D loss: 0.431734, acc.: 82.03%] [G loss: 1.061780]
epoch:29 step:27864 [D loss: 0.522731, acc.: 77.34%] [G loss: 1.169403]
epoch:29 step:27865 [D loss: 0.495103, acc.: 74.22%] [G loss: 0.607679]
epoch:29 step:27866 [D loss: 0.540144, acc.: 68.75%] [G loss: 1.298022]
epoch:29 step:27867 [D loss: 0.490200, acc.: 70.31%] [G loss: 0.

epoch:29 step:27969 [D loss: 0.360189, acc.: 81.25%] [G loss: 1.069861]
epoch:29 step:27970 [D loss: 1.044299, acc.: 36.72%] [G loss: 2.421162]
epoch:29 step:27971 [D loss: 0.880241, acc.: 54.69%] [G loss: 1.656352]
epoch:29 step:27972 [D loss: 1.086072, acc.: 50.78%] [G loss: 1.591732]
epoch:29 step:27973 [D loss: 0.743186, acc.: 57.81%] [G loss: 2.186439]
epoch:29 step:27974 [D loss: 0.675343, acc.: 59.38%] [G loss: 2.274911]
epoch:29 step:27975 [D loss: 0.296218, acc.: 87.50%] [G loss: 2.105376]
epoch:29 step:27976 [D loss: 0.497107, acc.: 71.88%] [G loss: 2.089083]
epoch:29 step:27977 [D loss: 0.230331, acc.: 95.31%] [G loss: 1.888317]
epoch:29 step:27978 [D loss: 0.294240, acc.: 92.19%] [G loss: 1.747784]
epoch:29 step:27979 [D loss: 0.117186, acc.: 99.22%] [G loss: 1.501822]
epoch:29 step:27980 [D loss: 0.824915, acc.: 53.12%] [G loss: 0.552280]
epoch:29 step:27981 [D loss: 0.437738, acc.: 80.47%] [G loss: 1.678262]
epoch:29 step:27982 [D loss: 0.600305, acc.: 64.84%] [G loss: 1.

epoch:29 step:28085 [D loss: 0.136549, acc.: 100.00%] [G loss: 1.914384]
epoch:29 step:28086 [D loss: 0.740362, acc.: 53.12%] [G loss: 1.513388]
epoch:29 step:28087 [D loss: 0.747407, acc.: 57.03%] [G loss: 1.243421]
epoch:29 step:28088 [D loss: 0.951154, acc.: 35.16%] [G loss: 1.172137]
epoch:29 step:28089 [D loss: 0.662386, acc.: 64.06%] [G loss: 1.292299]
epoch:29 step:28090 [D loss: 0.340367, acc.: 85.16%] [G loss: 1.480267]
epoch:29 step:28091 [D loss: 0.756401, acc.: 46.09%] [G loss: 1.229297]
epoch:29 step:28092 [D loss: 0.341574, acc.: 91.41%] [G loss: 1.482868]
epoch:29 step:28093 [D loss: 0.196378, acc.: 94.53%] [G loss: 1.379364]
epoch:29 step:28094 [D loss: 0.257433, acc.: 90.62%] [G loss: 1.331200]
epoch:29 step:28095 [D loss: 0.803379, acc.: 50.00%] [G loss: 1.533815]
epoch:29 step:28096 [D loss: 0.570679, acc.: 69.53%] [G loss: 1.052172]
epoch:29 step:28097 [D loss: 0.692455, acc.: 61.72%] [G loss: 1.280609]
epoch:29 step:28098 [D loss: 0.299978, acc.: 92.97%] [G loss: 1